In [ ]:
"""
Day 20 - Grove Positioning System

The input file is a list of numbers. To mix the file, move each number forward or backward in the file a number of positions equal to the value of the number being moved. The list is circular, so moving a number off one end of the list wraps back around to the other end as if the ends were connected.

For example, to move the 1 in a sequence like 4, 5, 6, 1, 7, 8, 9, the 1 moves one position forward: 4, 5, 6, 7, 1, 8, 9. To move the -2 in a sequence like 4, -2, 5, 6, 7, 8, 9, the -2 moves two positions backward, wrapping around: 4, 5, 6, 7, 8, -2, 9.

The numbers should be moved in the order they originally appear in the encrypted file. Numbers moving around during the mixing process do not change the order in which the numbers are moved.

Consider this encrypted file:

1
2
-3
3
-2
0
4
Mixing this file proceeds as follows:

Initial arrangement:
1, 2, -3, 3, -2, 0, 4

1 moves between 2 and -3:
2, 1, -3, 3, -2, 0, 4

2 moves between -3 and 3:
1, -3, 2, 3, -2, 0, 4

-3 moves between -2 and 0:
1, 2, 3, -2, -3, 0, 4

3 moves between 0 and 4:
1, 2, -2, -3, 0, 3, 4

-2 moves between 4 and 1:
1, 2, -3, 0, 3, 4, -2

0 does not move:
1, 2, -3, 0, 3, 4, -2

4 moves between -3 and 0:
1, 2, -3, 4, 0, 3, -2
Then, the grove coordinates can be found by looking at the 1000th, 2000th, and 3000th numbers after the value 0, wrapping around the list as necessary. In the above example, the 1000th number after 0 is 4, the 2000th is -3, and the 3000th is 2; adding these together produces 3.
"""

In [72]:
# create a circular linked list of the input numbers, and store an object reference to the nodes in the order they appear in the input file
class Node:
    def __init__(self, value, shift_amount, id):
        self.id = id
        self.value = value
        self.next = None
        self.prev = None
        self.shift_amt = shift_amount

    def move(self, n): 
        # move the node n positions forward or backward
        if n == 0:
            return
        insert_after = self
        if n > 0:
            for i in range(n):
                insert_after = insert_after.next
                if insert_after == self:
                    # skip over self on iterations
                    insert_after = insert_after.next
        else:
            for i in range(abs(n) + 1):
                insert_after = insert_after.prev
                if insert_after == self:
                    # skip over self on iterations
                    insert_after = insert_after.prev
        if self.prev == insert_after or self == insert_after:
            # print("Skipping move of {} to same spot".format(self.value))
            return
        self.remove()
        self.insert_after(insert_after)

    def remove(self):
        # remove the node from the list
        self.prev.next = self.next
        self.next.prev = self.prev

    def insert_after(self, node):
        # insert the node after the given node
        self.next = node.next
        self.prev = node
        node.next.prev = self
        node.next = self

    def neighbors(self, before_context, after_context):
        # print the nodes before and after the current node
        current = self
        neighbors = []
        for i in range(before_context):
            current = current.prev
        for i in range(before_context + after_context + 1):
            neighbors.append(current)
            current = current.next
        return neighbors

    def lookahead(self, n):
        # return the node n positions ahead
        current = self
        for i in range(n):
            current = current.next
        return current

    def __str__(self) -> str:
        return str(self.value)

class CircularDoublyLinkedList:

    def __init__(self) -> None:
        self.head = None
        self.tail = None
        self.size = 0

    def append(self, value, shift_amt, id):
        node = Node(value, shift_amt, id)
        if self.head is None:
            self.head = node
            self.tail = node
            node.next = node
            node.prev = node
        else:
            self.tail.next = node
            node.prev = self.tail
            node.next = self.head
            self.head.prev = node
            self.tail = node
        self.size += 1
        return node

    def find(self, value):
        current = self.head
        for i in range(self.size):
            if current.value == value:
                return current
            current = current.next
        return None

    def shift_to_head(self, value):
        # move the node with the given value to the head of the list
        node = self.find(value)
        if node is not None:
            self.head = node

    def __str__(self) -> str:
        current = self.head
        output = []
        for i in range(self.size):
            output.append(str(current))
            current = current.next
        return '[' + ', '.join(output) + ']'

    # iterator
    def __iter__(self):
        current = self.head
        for i in range(self.size):
            yield current
            current = current.next

In [2]:
test_cases = [
    {'test': [1, 2, -3, 3, -2, 0, 4], 'result': [0, 3, -2, 1, 2, -3, 4], 'test_description': 'Sample problem input' },
{ 'test': [0, 4, -5, -6, 7, 8], 'result': [0, 8, -6, -5, 4, 7], 'test_description': 'Test with positive and negative numbers wrapping around both ends of the list' },
{ 'test': [0, -11, 11, -11, 11, -11], 'result': [0, -11, 11, 11, -11, -11], 'test_description': 'Test with multiple large positive and negative numbers' },
{ 'test': [0, 1, 2, 3, 4, 5], 'result': [0, 1, 4, 2, 5, 3], 'test_description': 'Sequence shuffle with wrap' },
{ 'test': [0, -1, -2, -3, -4, -5], 'result': [0, -5, -4, -3, -2, -1], 'test_description': 'Test with all negative numbers' },
{ 'test': [0, 11, 11, 11, 11, 11], 'result': [0, 11, 11, 11, 11, 11], 'test_description': 'Test with all positive numbers' }
]

test_cases_2 = [
    {'test': [1, 2, -3, 3, -2, 0, 4], 'result': [0, -2434767459, 1623178306, 3246356612, -1623178306, 2434767459, 811589153], 'test_description': 'Sample problem input' },
]

In [78]:
def process_list(input_numbers, decrypt_key = 1, mix_count = 1):
    # create a circular doubly linked list of the input numbers
    numbers = CircularDoublyLinkedList()
    sequence = []
    list_len = len(input_numbers)
    for i, number in enumerate(input_numbers):
        keyed_number = number * decrypt_key
        
        shift_amount = keyed_number % (list_len - 1)

        # print(number, keyed_number, shift_amount)
        sequence.append(numbers.append(keyed_number, shift_amount, i))

    # move each number forward or backward in the list a number of positions equal to the value of the number being moved
    for x in range(mix_count):
        print("Mix round: {}".format(x + 1))
        for i, current in enumerate(sequence):
            # old_prev, old_next = current.prev, current.next
            # shift_distance = current.value
            shift_distance = current.shift_amt
            current.move(shift_distance)
            # print("Moving {} between {} and {} by {} steps, between {} and {}".format(current, old_prev, old_next, shift_distance, current.prev.value, current.next.value))
            # if current == numbers.head:
            #     numbers.head = old_next
            # print(numbers)
        # numbers.shift_to_head(0)
        # print(numbers)
    # numbers.shift_to_head(0)
    # print(numbers)

    return numbers
    

In [76]:
# decrypt_key = 1
decrypt_key = 811589153

# for test_case in test_cases:
test_case = test_cases_2[0]
input_numbers = [x * decrypt_key for x in test_case['test']]
expected_output = str(test_case['result'])
print("Test: {}".format(test_case['test_description']))
print('Input: {}'.format(input_numbers))
print('Expected: {}'.format(expected_output))
result = process_list(test_case['test'], decrypt_key, 10)
result.shift_to_head(0)
print("Result")
print(result)

zero = result.find(0)
zero1k = zero.lookahead(1000)
zero2k = zero.lookahead(2000)
zero3k = zero.lookahead(3000)
print("Zero -> 1000: {}, 2000: {}, 3000: {}, sum: {}".format(zero1k, zero2k, zero3k, zero1k.value + zero2k.value + zero3k.value))

Test: Sample problem input
Input: [811589153, 1623178306, -2434767459, 2434767459, -1623178306, 0, 3246356612]
Expected: [0, -2434767459, 1623178306, 3246356612, -1623178306, 2434767459, 811589153]
Mix round: 1
[0, -2434767459, 3246356612, -1623178306, 2434767459, 1623178306, 811589153]
Mix round: 2
[0, 2434767459, 1623178306, 3246356612, -2434767459, -1623178306, 811589153]
Mix round: 3
[0, 811589153, 2434767459, 3246356612, 1623178306, -1623178306, -2434767459]
Mix round: 4
[0, 1623178306, -2434767459, 811589153, 2434767459, 3246356612, -1623178306]
Mix round: 5
[0, 811589153, -1623178306, 1623178306, -2434767459, 3246356612, 2434767459]
Mix round: 6
[0, 811589153, -1623178306, 3246356612, -2434767459, 1623178306, 2434767459]
Mix round: 7
[0, -2434767459, 2434767459, 1623178306, -1623178306, 811589153, 3246356612]
Mix round: 8
[0, 1623178306, 3246356612, 811589153, -2434767459, 2434767459, -1623178306]
Mix round: 9
[0, 811589153, 1623178306, -2434767459, 3246356612, 2434767459, -1623

In [44]:
for x in range(-7, 7, 3):
    test_list = CircularDoublyLinkedList()
    test_list.append(1, 1, 1)
    test_list.append(2, 2, 2)
    test_list.append(3, 3, 3)
    test_node = test_list.append(0, x, 0)
    test_list.append(4, 4, 4)
    test_list.append(5, 5, 5)
    test_list.append(6, 6, 6)

    print('  ', test_list, ' len: ', test_list.size)
    test_node.move(test_node.shift_amt)
    # print x and test list with x padded to 2 spaces
    print('{:2d}'.format(x), test_list)
    print()
    

   [1, 2, 3, 0, 4, 5, 6]  len:  7
-7 [1, 2, 0, 3, 4, 5, 6]

   [1, 2, 3, 0, 4, 5, 6]  len:  7
-4 [1, 2, 3, 4, 5, 0, 6]

   [1, 2, 3, 0, 4, 5, 6]  len:  7
-1 [1, 2, 0, 3, 4, 5, 6]

   [1, 2, 3, 0, 4, 5, 6]  len:  7
 2 [1, 2, 3, 4, 5, 0, 6]

   [1, 2, 3, 0, 4, 5, 6]  len:  7
 5 [1, 2, 0, 3, 4, 5, 6]



In [79]:
input_numbers = [int(line) for line in open('day20-input.txt')]
result = process_list(input_numbers, 811589153, 10)
# print(result)
zero = result.find(0)
print(zero)
la1000 = zero.lookahead(1000)
la2000 = zero.lookahead(2000)
la3000 = zero.lookahead(3000)
print(la1000, la2000, la3000)
print(la1000.value + la2000.value + la3000.value)

Mix round: 1
Mix round: 2
Mix round: 3
Mix round: 4
Mix round: 5
Mix round: 6
Mix round: 7
Mix round: 8
Mix round: 9
Mix round: 10
0
6329583804247 7175259701673 -7084361716537
6420481789383
